# Model Monitoring with Evidently and MLFlow

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import pandas as pd
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from pathlib import Path
from sklearn import ensemble
from typing import Dict, Tuple

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

from src.reports import (
    build_model_monitoring_report,
    get_model_monitoring_metrics
)

from config import MLFLOW_TRACKING_URI, DATA_DIR, FILENAME, REPORTS_DIR

/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnr

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [3]:
# Download original dataset with: python src/pipelines/load_data.py 
raw_data = pd.read_csv(f"../{DATA_DIR}/{FILENAME}")

# Set datetime index 
raw_data = raw_data.set_index('dteday')


raw_data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Define column mapping

In [4]:
target = 'cnt'
prediction = 'prediction'
datetime = 'dteday'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]
FEATURE_COLUMNS = numerical_features + categorical_features

column_mapping = ColumnMapping()
column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.datetime = datetime
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

## Load Model (from the MLFlow)

In [5]:
# Update MLFLOW_RUN_ID 

# Copy a Run ID from the "Model Testing" experiment
MLFLOW_RUN_ID = 'ae7a93fb0eb44e69a46c359d9657364a'

# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Get 
model_path = mlflow.artifacts.download_artifacts(
    run_id=MLFLOW_RUN_ID, 
    artifact_path='model.joblib'
)
print("Model path: ", model_path)

Client tracking uri: http://localhost:5000
Model path:  /var/folders/ff/tyq6m7sn6gj89qfmv0lxbgc00000gn/T/tmpmc8m9hyk/model.joblib


In [6]:
# Load Model

regressor = joblib.load(model_path)

regressor

RandomForestRegressor(n_estimators=50, random_state=0)

# Define dates for train and inference

In [7]:
# Define dates for train data
train_dates = ('2011-01-02 00:00:00','2011-03-06 23:00:00')

# Define dates for inference batches
prediction_batches = [ 
    ('2011-03-07 00:00:00','2011-03-13 23:00:00'),
    ('2011-03-14 00:00:00','2011-03-20 23:00:00'),
    ('2011-03-21 00:00:00','2011-03-27 23:00:00'), 
]

## Define the Reference data

In [8]:
# Define the reference dataset
reference_data = raw_data.loc[train_dates[0]:train_dates[1]]
reference_prediction = regressor.predict(reference_data[FEATURE_COLUMNS])
reference_data['prediction'] = reference_prediction

print(reference_data.shape)

(1433, 17)


# Monitor Model

### Week 1

In [9]:
current_dates = prediction_batches[0]
current_data = raw_data.loc[current_dates[0]:current_dates[1]]  

print(current_data.shape)
# current_data.head()

(140, 16)


In [10]:
current_prediction = regressor.predict(current_data[numerical_features + categorical_features])
current_data['prediction'] = current_prediction

print(current_data.shape)

(140, 17)


In [11]:
model_report = build_model_monitoring_report(
    reference_data=reference_data.reset_index(drop=True),
    current_data=current_data.reset_index(drop=True),
    column_mapping=column_mapping,
)

model_metrics = get_model_monitoring_metrics(model_report)
model_metrics

{'me': 4.652285714285715,
 'mae': 25.00857142857143,
 'rmse': 1565.6536971428573,
 'mape': 86.72439477100151}

### Week 2

In [12]:
current_dates = prediction_batches[1]
current_data = raw_data.loc[current_dates[0]:current_dates[1]]  

current_prediction = regressor.predict(current_data[numerical_features + categorical_features])
current_data['prediction'] = current_prediction

print(current_dates)
print(current_data.shape)
# current_data.head()

('2011-03-14 00:00:00', '2011-03-20 23:00:00')
(141, 17)


In [13]:
model_report = build_model_monitoring_report(
    reference_data=reference_data.reset_index(drop=True),
    current_data=current_data.reset_index(drop=True),
    column_mapping=column_mapping,
)

model_metrics = get_model_monitoring_metrics(model_report)
model_metrics

{'me': -23.349645390070922,
 'mae': 30.37035460992908,
 'rmse': 2384.702697872341,
 'mape': 33.42546040174057}

# Model Quality Evaluation (Prod)

In [14]:
# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment("Monitor Model")

2023/07/11 18:21:58 INFO mlflow.tracking.fluent: Experiment with name 'Monitor Model' does not exist. Creating a new experiment.


Client tracking uri: http://localhost:5000


<Experiment: artifact_location='mlflow-artifacts:/888387858294031661', creation_time=1689092518430, experiment_id='888387858294031661', last_update_time=1689092518430, lifecycle_stage='active', name='Monitor Model', tags={}>

In [15]:

# Run model monitoring for each batch of dates
for current_dates in prediction_batches:
    
    print(f"Current batch dates: {current_dates}") 

    # Start a new Run for the batch
    with mlflow.start_run(run_name=current_dates[1]) as run: 
        
        # Show newly created run metadata info
        print("Experiment id: {}".format(run.info.experiment_id))
        print("Run id: {}".format(run.info.run_id))
        print("Run name: {}".format(run.info.run_name))
            
        # Log parameters
        mlflow.log_param("begin", current_dates[0])
        mlflow.log_param("end", current_dates[1])
        
        # Make predictions for the current batch data
        current_data = raw_data.loc[current_dates[0]:current_dates[1]]
        current_prediction = regressor.predict(current_data[FEATURE_COLUMNS])
        current_data['prediction'] = current_prediction

        # Build the Model Monitoring report
        model_report = build_model_monitoring_report(
            reference_data=reference_data.reset_index(drop=True),
            current_data=current_data.reset_index(drop=True),
            column_mapping=column_mapping,
        )
        
        # Log Metrics
        model_metrics = get_model_monitoring_metrics(model_report)
        mlflow.log_metrics(model_metrics)
        
        # Log Monitoring Report 
        monitoring_report_path = f"../{REPORTS_DIR}/model_monitoring_report.html"
        model_report.save_html(monitoring_report_path)
        mlflow.log_artifact(monitoring_report_path)
        
        print(run.info)

Current batch dates: ('2011-03-07 00:00:00', '2011-03-13 23:00:00')
Experiment id: 888387858294031661
Run id: a14082fd547848ed8719d906b7eeb49b
Run name: 2011-03-13 23:00:00
<RunInfo: artifact_uri='mlflow-artifacts:/888387858294031661/a14082fd547848ed8719d906b7eeb49b/artifacts', end_time=None, experiment_id='888387858294031661', lifecycle_stage='active', run_id='a14082fd547848ed8719d906b7eeb49b', run_name='2011-03-13 23:00:00', run_uuid='a14082fd547848ed8719d906b7eeb49b', start_time=1689092518602, status='RUNNING', user_id='mnrozhkov'>
Current batch dates: ('2011-03-14 00:00:00', '2011-03-20 23:00:00')
Experiment id: 888387858294031661
Run id: d33172ade6464ef18a830aa99276ea63
Run name: 2011-03-20 23:00:00
<RunInfo: artifact_uri='mlflow-artifacts:/888387858294031661/d33172ade6464ef18a830aa99276ea63/artifacts', end_time=None, experiment_id='888387858294031661', lifecycle_stage='active', run_id='d33172ade6464ef18a830aa99276ea63', run_name='2011-03-20 23:00:00', run_uuid='d33172ade6464ef18a